In [1]:
import datetime as dt
import logging
import json
from pathlib import Path

import duckdb
import pandas as pd

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

data_dir = Path("/projects/dsgrid/tempo-project/april-queries/")
dataset_name = "full_dataset"

## DuckDB and Partitioned File Utilities

In [2]:
def is_partitioned(filepath):
    for p in filepath.iterdir():
        if p.is_dir() and ("=" in p.stem) and (len(p.stem.split("=")) == 2):
            return True
    return False

def get_partitions(filepath):
    assert is_partitioned(filepath), f"{filepath} is not partitioned"
    
    partition_name = None
    for p in filepath.iterdir():
        if p.is_dir() and ("=" in p.stem):
            tmp, value = p.stem.split("=")
            if partition_name:
                assert (tmp == partition_name), f"Found two different partition names in {filepath}: {partition_name}, {tmp}"
            partition_name = tmp
            yield partition_name, value, p

def print_partitions(filepath, print_depth=2, _depth=0):
    if is_partitioned(filepath):
        space = ' ' * 4 * _depth
        for partition_name, value, p in get_partitions(filepath):
            print(f"{space}{partition_name}={value}")
        if (not print_depth) or ((_depth + 1) < print_depth):
            print_partitions(p, print_depth=print_depth, _depth=_depth+1)

def load_table(filepath, tablename):
    duckdb.sql(f"CREATE TABLE {tablename} AS SELECT * FROM read_parquet('{filepath}/**/*.parquet', hive_partitioning=true, hive_types_autocast=false);")
    description = duckdb.sql(f"DESCRIBE {tablename};")
    logger.info(f"Loaded {filepath} as {tablename}:\n{description}")

## Load Data

In [3]:
def get_metadata(dataset_path):
    with open(dataset_path / "metadata.json") as f:
        result = json.load(f)
    return result

# load metadata and get column names by type
metadata = get_metadata(data_dir / dataset_name)
assert metadata["table_format"]["format_type"] == "unpivoted", metadata["table_format"]
value_column = metadata["table_format"]["value_column"]
columns_by_type = {dim_type: metadata["dimensions"][dim_type][0]["column_names"][0] for dim_type in metadata["dimensions"]}

In [ ]:
# load data table
filepath = data_dir / dataset_name / "table.parquet"
tablename = "tbl"
load_table(filepath, tablename)
duckdb.sql(f"SELECT * FROM {tablename} LIMIT 5").df()

In [ ]:
count = duckdb.sql(f"SELECT COUNT(*) as count FROM {tablename};").df()["count"].values[0]
print(f"Dataset contains {count:,} data points")

## Recreate Lefthand Side of Figure ES-1

In [ ]:
df = duckdb.sql(f"""SELECT scenario, {columns_by_type["model_year"]} as year, SUM({value_column})/1.0E6 as annual_twh
                      FROM {tablename} 
                  GROUP BY scenario, {columns_by_type["model_year"]}
                  ORDER BY scenario, year""").df()
df["scenario"] = df["scenario"].map({
    "efs_high_ldv": "EFS High Electrification",
    "ldv_sales_evs_2035": "All LDV Sales EV by 2035",
    "reference": "AEO Reference"
})

In [ ]:
import plotly.express as px

fig = px.line(df, x="year", y="annual_twh", color="scenario", 
              labels={"annual_twh": "EV Load (TWh/yr)", "scenario": "Scenario"}, 
              range_y=[-25,1025],
              width=600, height=450, template="plotly_white")
fig

## Verify Timestamps Are As Expected

In [ ]:
assert columns_by_type["time"] == "time_est", "Code in this section only makes sense if the dataset has timestamps"

In [ ]:
# select a subset of the data and look at initial timestamps

where_clause = f"(scenario = 'reference') AND ({columns_by_type['model_year']} = 2050)"

if columns_by_type['geography'] == "census_division":
    where_clause += f" AND ({columns_by_type['geography']} = 'middle_atlantic')"
elif columns_by_type['geography'] == "state":
    where_clause += f" AND ({columns_by_type['geography']} = 'RI')"
else:
    raise NotImplementedError()

if columns_by_type['subsector'] == "subsector":
    where_clause += f" AND ({columns_by_type['subsector']} = 'bev_compact')"
else:
    raise NotImplementedError()

duckdb.sql(f"SELECT * FROM {tablename} WHERE {where_clause} ORDER BY time_est LIMIT 5;")

**Because the data type is "TIMESTAMP" and the first timestamp listed is "2012-01-01 05:00:00", we can see that DuckDB is importing time_est as timezone-unaware timestamps that are in UTC.**

In [ ]:
# demonstrate how to shift timestamps to match 'time_est' label

columns_clause = ""; sep= ""
for column in duckdb.sql(f"DESCRIBE {tablename};").df()["column_name"].tolist():
    if column == "time_est":
        columns_clause += sep + "time_est - INTERVAL 5 HOUR as time_est"
    else:
        columns_clause += sep + column
    sep = ", "

sql_stmt = f"""
    SELECT {columns_clause} 
      FROM {tablename} 
     WHERE {where_clause} 
  ORDER BY time_est 
     LIMIT 5;"""
logger.info(f"SQL for adjusting timestamps:\n{sql_stmt}")
duckdb.sql(sql_stmt)

## Verify that Profiles in Different Timezones Are As Expected

In [ ]:
assert columns_by_type["time"] == "time_est", "Code in this section only makes sense if the dataset has timestamps"

In [ ]:
def get_profile(start_timestamp, end_timestamp, where_clause, 
                tablename=tablename, value_column=value_column, 
                normalize_profile=True, replace_timestamps=True):
    df = duckdb.sql(f"""SELECT time_est, SUM({value_column}) as {value_column}
                          FROM {tablename} 
                         WHERE {where_clause} AND (time_est >= TIMESTAMP '{start_timestamp}') AND (time_est <= TIMESTAMP '{end_timestamp}')
                      GROUP BY time_est 
                      ORDER BY time_est;""").df()
    if normalize_profile:
        df[value_column] = df[value_column] / df[value_column].sum()
    if replace_timestamps:
        df["hour"] = df.index.values
        df = df[["hour",value_column]]
    return df

where_clause = f"(scenario = 'reference') AND ({columns_by_type['model_year']} = 2050)"

if columns_by_type['subsector'] == "subsector":
    where_clause += f" AND ({columns_by_type['subsector']} = 'bev_compact')"
else:
    raise NotImplementedError()

geographies = None
if columns_by_type['geography'] == "census_division":
    geographies = {"ET": "middle_atlantic", "CT": "west_south_central", "MT": "mountain", "PT": "pacific"}
elif columns_by_type['geography'] == "state":
    geographies = {"ET": "NC", "CT": "TX", "MT": "CO", "PT": "OR"}
else:
    raise NotImplementedError()

days = {
    "Standard Time": (dt.datetime(2012, 2, 14, 5), dt.datetime(2012, 2, 15, 4)),        # Selects UTC timestamps that correspond to EST 2/14/2012
    "Daylight Savings Time": (dt.datetime(2012, 8, 14, 5), dt.datetime(2012, 8, 15, 4)) # Selects UTC timestamps that correspond to EST 8/14/2012
}

data = []
for time_type, time_tuple in days.items():
    for tz, geo in geographies.items():
        data.append(get_profile(time_tuple[0], time_tuple[1], where_clause + f" AND {columns_by_type['geography']} = '{geo}'"))
        data[-1]["Time Type"] = time_type
        data[-1]["Time Zone"] = tz
df = pd.concat(data)
df

In [ ]:
import plotly.express as px

fig = px.line(df, x="hour", y=value_column, color="Time Zone", line_dash="Time Type",
              color_discrete_map={"ET": "red", "CT": "orange", "MT": "blue", "PT": "purple"},
              labels={"value": "Normalized Load Profile", "hour": "Hour of EST Day"},
              #range_y=[0,0.1],
              width=600, template="plotly_white")
fig

## Demonstrate Loading a Subset of a Larger Dataset

In [4]:
filepath = data_dir / dataset_name / "table.parquet"
print_partitions(filepath, print_depth=None)

scenario=efs_high_ldv
scenario=ldv_sales_evs_2035
scenario=reference
    tempo_project_model_years=2024
    tempo_project_model_years=2046
    tempo_project_model_years=2030
    tempo_project_model_years=2044
    tempo_project_model_years=2026
    tempo_project_model_years=2036
    tempo_project_model_years=2042
    tempo_project_model_years=2050
    tempo_project_model_years=2028
    tempo_project_model_years=2048
    tempo_project_model_years=2040
    tempo_project_model_years=2038
    tempo_project_model_years=2032
    tempo_project_model_years=2034
        state=MA
        state=KY
        state=NH
        state=MI
        state=CT
        state=NE
        state=IL
        state=MD
        state=WY
        state=RI
        state=CA
        state=MT
        state=MN
        state=TN
        state=NJ
        state=NC
        state=ID
        state=IN
        state=ND
        state=WV
        state=WI
        state=MS
        state=CO
        state=SC
        state=KS
        state=NY

In [ ]:
# Edit this list of tuples as desired
partitions=[
    ("scenario", "efs_high_ldv"),
    ("tempo_project_model_years", "2050")
]

subset_filepath = filepath
for partition_name, value in partitions:
    subset_filepath = subset_filepath / f"{partition_name}={value}"

assert subset_filepath.exists(), f"{subset_filepath} does not exist. Edit `partitions`."

# load partial data table
tablename = "tbl"
load_table(subset_filepath, tablename)
duckdb.sql(f"SELECT * FROM {tablename} LIMIT 5").df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
count = duckdb.sql(f"SELECT COUNT(*) as count FROM {tablename};").df()["count"].values[0]
print(f"Dataset contains {count:,} data points")

In [ ]:
# TODO: Add delete, insert